# Simulated Probit

In [34]:
import numpy as np
import gurobipy as grb

In [35]:
seed = 777
nbDraws = 10
U_y = np.array([0.4, 0.5, 0.2, 0.3, 0.1, 0])
nbY = len(U_y)

In [36]:
rho = 0.5
Covar = rho * np.ones((nbY, nbY)) + (1 - rho) * np.eye(nbY)

In [37]:
E = np.linalg.eigh(Covar)
V = E[0]
Q = E[1]
SqrtCovar = Q.dot(np.diag(np.sqrt(V))).dot(Q.T)

In [38]:
epsilon_iy = np.random.normal(0,1,nbDraws*nbY).reshape(nbDraws,nbY).dot(SqrtCovar)
u_iy = epsilon_iy + U_y

In [39]:
ui = np.max(u_iy, axis=1)
s_y = np.sum((u_iy.T - ui).T == 0, axis=0) / nbDraws

In [40]:
opt_assign = [(i, j) for i in range(nbY) for j in range(nbDraws)]

m = grb.Model()
vars = m.addVars(opt_assign, obj=epsilon_iy.T.ravel(), name='vars')
m.ModelSense = -1
m.addConstrs(vars.sum('*', i) == 1/nbDraws for i in range(nbDraws))
m.addConstrs(vars.sum(i, '*') == s_y[i] for i in range(nbY))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [41]:
m.optimize()
if m.Status == grb.GRB.Status.OPTIMAL:
    pi = m.getAttr('pi')
    Uhat_y = -np.subtract(pi[nbDraws:nbY+nbDraws], pi[nbY + nbDraws - 1])
    print('U_y (true and recovered)')
    print(U_y)
    print(Uhat_y)

Optimize a model with 16 rows, 60 columns and 120 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-05, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 4e-01]
Presolve time: 0.01s
Presolved: 16 rows, 60 columns, 120 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4480847e+31   7.400000e+31   2.448085e+01      0s
      17    1.0047726e+00   0.000000e+00   0.000000e+00      0s

Solved in 17 iterations and 0.01 seconds
Optimal objective  1.004772613e+00
U_y (true and recovered)
[0.4 0.5 0.2 0.3 0.1 0. ]
[ 0.20724721  0.51684279  0.42056882  0.25652241 -0.30821205 -0.        ]
